In [ ]:
# import packages
import re,os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from IPython.display import clear_output
import pandas as pd
import math
import random
from datetime import datetime
from scipy.stats import linregress
from tqdm import trange
tfk = tf.keras
tfkl = tf.keras.layers
clear_output()
# import my files
from parameters import *
from MRA_generate import MRA_generate
from symae_model import SymAE
from plot_training import plot_training
from plot_redatuming import plot_redatuming
from plot_save import plot_save
from CustomCallback import CustomCallback
from redatuming import redatuming
from latent import latent 

In [ ]:
# Generate the Dataset
def g(n,x):
    if n==0:
        return math.e**(-9*x**2)
    elif n==1:
        return int(x<0.5)
    elif n==2:
        return int(x<0.2)
    elif n==3:
        return math.cos(2*math.pi*x)
    elif n==4:
        return math.cos(8*math.pi*x)
    else:
        return np.inf
MRA_data=MRA_generate(d,nt,N,sigma,ne,g,replace=False,outer_replace=True)
X=MRA_data.X
print("Numbers of States:")
pd.DataFrame(MRA_data.states).value_counts()

In [ ]:
def find_state(state):
    MRA_data=MRA_generate(d,nt,1,sigma,ne,g)
    for i in range(100):
        MRA_data.generate_default()
        if MRA_data.states[0]==state:
            return MRA_data
def alter_parameters(p,q):
    line_storage=''
    with open('parameters.py','r+') as f:
        for line in f.readlines():
            if(line.find('p') == 0):
                line = 'p=%d\n'%p
            if(line.find('q') == 0):
                line = 'q=%d\n'%q
            line_storage += line
    with open('parameters.py','r+') as f:
        f.writelines(line_storage)

In [ ]:
auto_path = './pq_checkpoint/status.txt'
auto_status = 'empty'

In [ ]:
def everything(p,q):
    alter_parameters(p,q)
    model=SymAE(N,nt,d,p,q,kernel_size,filters,dropout_rate)
    Adam=tf.keras.optimizers.Adam(learning_rate=0.001,beta_1=0.9,beta_2=0.999,epsilon=1e-07)
    model.compile(loss='mse',optimizer=Adam)
    try: 
        model.load_weights('./pq_checkpoint/'+'p=%d,q=%d'%(p,q))
    except:
        print('create '+'p=%d,q=%d'%(p,q))
    M=500
    epochs=range(M)
    losses=[0.0]*M
    slope=0.0; intercept=0.0; std_err=0.0
    loop_time = 0
    class CustomCallback(tf.keras.callbacks.Callback):
        def on_epoch_end(self, epoch, logs=None):
            clear_output(wait=True)
            losses[epoch]=logs["loss"]
            print("p={:d}, q={:d}. For epoch {:d}, loss is {:f}.".format(p,q,epoch,logs["loss"]))
            if epoch%100==0:
                auto_status="p={:d}, q={:d}. loop_time={:d}. For epoch {:d}, loss is {:f}.".format(p,q,loop_time,epoch,logs["loss"])
                auto_f = open(auto_path,'w')
                print(auto_status,file=auto_f)
                auto_f.close()
    while True:
        history=model.fit(X,X,epochs=M,verbose=0,callbacks=[CustomCallback()])
        model.save_weights('./pq_checkpoint/'+'p=%d,q=%d'%(p,q))
        slope, intercept, _, _, std_err = linregress(epochs[-201:-1], losses[-201:-1])
        if slope+std_err>0 and losses[-1]<0.02 :
            break
        if loop_time > 50:
            break
        if loop_time > 20 and losses[-1] > 0.1 :
            return False
        loop_time += 1
        # Epoch-Loss
        plt.plot(epochs,losses)
        plt.plot(epochs,intercept+slope*np.array(epochs))
        plt.xlabel('epoch')
        plt.ylabel('loss')
        plt.tight_layout()
        fig = plt.gcf()
        fig.savefig('./pq_checkpoint/p=%d,q=%d,epoch-loss.png'%(p,q))
        # Redatuming
        MRA1=find_state(0)
        MRA2=find_state(3)
        redatum=redatuming(model,MRA1,MRA2,1)
        fig=plot_redatuming(redatum)
        plt.tight_layout()
        fig.savefig('./pq_checkpoint/p=%d,q=%d,cos_tri.png'%(p,q))
        # Redatuming
        MRA1=find_state(0)
        MRA2=find_state(1)
        redatum=redatuming(model,MRA1,MRA2,1)
        fig=plot_redatuming(redatum)
        plt.tight_layout()
        fig.savefig('./pq_checkpoint/p=%d,q=%d,squ_tri.png'%(p,q))
    return True

In [ ]:
for p in range(1,20,1):
    for q in range(1,30,1):
        print('traning p=%d q=%d\n'%(p,q))
        flag=everything(p,q)
        while flag==False:
            flag=everything(p,q)